In [1]:
import os
import json
import pandas as pd
import numpy as np
import re

In [514]:
file_path = 'id_03700_03750.csv'
data = pd.read_csv(file_path)
columns_to_remove = ['error', 'error_code', 'warning', 'warning_code', 'timestamp']
data = data.drop(columns=columns_to_remove, errors='ignore')
# Display the first few rows of the dataset to understand its structure
data.head()

,id,name,city,country_code,position,about,posts,groups,current_company,experience,...,organizations,location,input_url,linkedin_id,activity,linkedin_num_id,banner_image,honors_and_awards,similar_profiles,input
0,ravindrapratapsingh,Ravindra Pratap Singh,"Mumbai, Maharashtra, India",IN,"Tehsildar and Executive Magistrate, Govt of Ra...","I've worn many hats in my career- Director, Wr...",NaN,NaN,"{""link"":""https://in.linkedin.com/company/gover...","[{""title"":""तहसीलदार एवं कार्यपालक मजिस्ट्रेट"",...",...,"[{""title"":""AnkiCo Eduventure Private Limited"",...",Mumbai,https://www.linkedin.com/in/ravindrapratapsingh,ravindrapratapsingh,"[{""interaction"":""Liked by Ravindra Pratap Sing...",226124177,https://media.licdn.com/dms/image/v2/C5116AQF8...,"[{""title"":""Global Young Leader"",""publication"":...","[{""url"":""https://in.linkedin.com/in/advitiya-s...","{""url"":""https://www.linkedin.com/in/ravindrapr..."
1,swapnil-umredkar-5905a6136,Swapnil Umredkar CSM®,"Pune, Maharashtra, India",IN,Project Manager II Project-Program Management ...,🔰Project Manager | 🎯 Certified ScrumMaster (CS...,NaN,NaN,"{""link"":""https://in.linkedin.com/company/wyset...","[{""title"":""Project Manager"",""description_html""...",...,"[{""title"":""American Society of Mechanical Engi...",Pune,https://www.linkedin.com/in/swapnil-umredkar-5...,swapnil-umredkar-5905a6136,"[{""interaction"":""Liked by Swapnil Umredkar CSM...",556855740,https://media.licdn.com/dms/image/v2/C5616AQFC...,"[{""title"":""Third prize at ASME AEROMANIA at AS...","[{""url"":""https://in.linkedin.com/in/-meetjain""...","{""url"":""https://www.linkedin.com/in/swapnil-um..."
2,swati-chandra-83b189,Swati Chandra,"New York, New York, United States",US,NaN,Seasoned investor who harnesses quant tools su...,NaN,NaN,"{""link"":""https://www.linkedin.com/company/capi...",NaN,...,NaN,New York,https://www.linkedin.com/in/swati-chandra-83b189,swati-chandra-83b189,NaN,431471,https://static.licdn.com/aero-v1/sc/h/5q92mjc5...,NaN,"[{""url"":""https://www.linkedin.com/in/ivonenin""...","{""url"":""https://www.linkedin.com/in/swati-chan..."
3,nishankmagoo,Nishank Magoo,"Pune, Maharashtra, India",IN,NaN,Nishank is an accomplished technologist with e...,NaN,NaN,"{""link"":""https://in.linkedin.com/company/mphas...",NaN,...,NaN,Pune,https://www.linkedin.com/in/nishankmagoo,nishankmagoo,"[{""interaction"":""Liked by Nishank Magoo"",""link...",102449782,https://media.licdn.com/dms/image/v2/D4D16AQGT...,"[{""title"":""The President Award"",""publication"":...","[{""url"":""https://in.linkedin.com/in/anita-bhoj...","{""url"":""https://www.linkedin.com/in/nishankmag..."
4,vivek-beniwal-888603195,Vivek Beniwal,"Hanumangarh, Rajasthan, India",IN,NaN,Experienced in driving digital growth and opti...,NaN,NaN,"{""link"":""https://ae.linkedin.com/company/get-d...",NaN,...,NaN,Hanumangarh,https://www.linkedin.com/in/vivek-beniwal-8886...,vivek-beniwal-888603195,"[{""interaction"":""Liked by Vivek Beniwal"",""link...",768897320,https://media.licdn.com/dms/image/v2/D5616AQGm...,NaN,"[{""url"":""https://in.linkedin.com/in/praveen-ku...","{""url"":""https://www.linkedin.com/in/vivek-beni..."


In [515]:
output_dir = 'json_files/'
os.makedirs(output_dir, exist_ok=True)
similar_profiles_dir = 'similar_profiles/'
os.makedirs(similar_profiles_dir, exist_ok=True)

In [516]:
columns_to_check = ['posts', 'current_company', 'experience', 'people_also_viewed', 'languages', 'recommendations', 'education', 'activity', 'certifications', 'volunteer_experience', 'organizations', 'honors_and_awards', 'similar_profiles', 'input']

# Display examples for potential restructuring
column_samples = {col: data[col].dropna().iloc[:5].tolist() for col in columns_to_check}
column_samples

{'posts': ['[{"title":"100 Habits of Highly Successful People: The Ultimate Guide to Crushing It (Without Crushing Yourself)","img":"https://media.licdn.com/dms/image/v2/D4D12AQEmsolU7BYB2w/article-cover_image-shrink_720_1280/article-cover_image-shrink_720_1280/0/1680409444356?e=2147483647&v=beta&t=p136_igRKESOjr21-JFDsUz5PTEpER0Tyz1W8nC76FQ","link":"https://www.linkedin.com/pulse/100-habits-highly-successful-people-ultimate-guide-crushing-daga","created_at":"2023-04-02T00:00:00.000Z","interaction":null,"id":"daga"},{"title":"Corporate Climbing 101: How to Reach the Top (Without Stepping on Anyone\'s Toes) 💼💪","img":"https://media.licdn.com/dms/image/v2/D4D12AQHgGpE6NDgHPw/article-cover_image-shrink_720_1280/article-cover_image-shrink_720_1280/0/1680314405676?e=2147483647&v=beta&t=E8aB0gw4oeOaSQdljBzh6ksL3yjOWTXIhH_1BgD9oAA","link":"https://www.linkedin.com/pulse/corporate-climbing-101-how-reach-top-without-stepping-nikhil-daga","created_at":"2023-04-01T00:00:00.000Z","interaction":nul

In [517]:
def parse_column(data, column_name):
    """Parse a column containing JSON-like strings into structured Python objects."""
    try:
        if pd.notnull(data[column_name]):
            return json.loads(data[column_name])
    except (json.JSONDecodeError, TypeError):
        pass
    return data[column_name]

# Update each row to structure specific columns
structured_columns = columns_to_check

In [518]:
def is_link(value):
    """Check if the value is a URL or link."""
    return isinstance(value, str) and (value.startswith('http://') or value.startswith('https://'))

In [519]:
def replace_null_with_none(value):
    """Recursively replace np.nan or NULL-like values with None."""
    if np.isscalar(value):  # Check if the value is scalar
        return None if pd.isnull(value) else value
    elif isinstance(value, list):  # Handle lists
        return [replace_null_with_none(v) for v in value]
    elif isinstance(value, dict):  # Handle dictionaries
        return {k: replace_null_with_none(v) for k, v in value.items()}
    return value

In [520]:
def remove_urls(text):
    """Remove URLs from a string."""
    if isinstance(text, str):
        # Regular expression to match URLs
        url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        return re.sub(url_pattern, '', text)
    return text

In [521]:
def remove_br_tags(text):
    """Remove <br> tags from a string."""
    if isinstance(text, str):
        return re.sub(r'<br\s*/?>', '', text)
    return text

In [522]:
def structure_and_clean_data(record, structured_columns):
    """Parse, structure, and clean data for the specified columns."""
    for column in structured_columns:
        if column in record:
            # Parse and structure the column
            structured_data = parse_column(record, column)
            
            # Remove URLs and <br> tags within structured data
            if isinstance(structured_data, list):
                record[column] = [
                    # If the item is a dictionary, process its items; otherwise, apply transformations directly
                    {k: remove_br_tags(remove_urls(v)) if isinstance(v, str) else v for k, v in item.items()} if isinstance(item, dict) else remove_br_tags(remove_urls(item))
                    for item in structured_data
                ]
            elif isinstance(structured_data, dict):
                record[column] = {
                    k: remove_br_tags(remove_urls(v)) if isinstance(v, str) else v
                    for k, v in structured_data.items()
                }
            else:
                record[column] = remove_br_tags(remove_urls(structured_data)) if isinstance(structured_data, str) else structured_data
    
    # Remove key-value pairs where the value is a link or contains <br> tags
    # If the value is a link, we remove both the key and the value
    record = {k: v for k, v in record.items() if not (is_link(v) or isinstance(v, str) and "<br>" in v)}
    
    return record

In [523]:
for _, row in data.iterrows():
    record_id = row['id']
    record_data = row.to_dict()
    
    # Structure and clean data in structured_columns
    record_data = structure_and_clean_data(record_data, structured_columns)
    
    # Extract and save similar_profiles to a separate folder
    similar_profiles = record_data.pop('similar_profiles', None)
    if similar_profiles:
        # Ensure the folder exists
        os.makedirs(similar_profiles_dir, exist_ok=True)
        
        # Filepath for the similar_profiles JSON file
        similar_profiles_path = os.path.join(similar_profiles_dir, f"{record_id}_similar_profiles.json")
        
        # Save similar_profiles data
        with open(similar_profiles_path, 'w', encoding='utf-8') as f:
            json.dump(similar_profiles, f, ensure_ascii=False, indent=4)
    
    # Replace None or null-like values with None (JSON-compatible null)
    record_data = {key: replace_null_with_none(value) for key, value in record_data.items()}
    
    # Filepath for the remaining data (excluding similar_profiles)
    json_file_path = os.path.join(output_dir, f"{record_id}.json")
    
    # Save remaining data to JSON file
    with open(json_file_path, 'w', encoding='utf-8') as f:
        json.dump(record_data, f, ensure_ascii=False, indent=4)

output_dir

'json_files/'

In [7]:
def remove_empty_string_keys(data):
    """Remove keys with empty string values."""
    if isinstance(data, dict):
        # Create a new dictionary without empty string values
        return {k: remove_empty_string_keys(v) for k, v in data.items() if v != "" or k != "input"}
    elif isinstance(data, list):
        # Apply to each item in the list
        return [remove_empty_string_keys(item) for item in data]
    else:
        # Return the data as is if it's neither a dict nor a list
        return data

In [8]:
def process_json_files(input_folder):
    """Process all JSON files in the given folder to remove empty string values."""
    for filename in os.listdir(input_folder):
        if filename.endswith(".json"):
            file_path = os.path.join(input_folder, filename)
            
            # Read the JSON data from the file
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # Remove keys with empty string values
            cleaned_data = remove_empty_string_keys(data)
            
            # Save the cleaned data back to the file
            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(cleaned_data, f, ensure_ascii=False, indent=4)

In [9]:
input_folder = "json_files"  # Replace with your folder path
process_json_files(input_folder)

In [10]:
input_folder = "similar_profiles"  # Replace with your folder path
process_json_files(input_folder)

In [11]:
def remove_empty_input(data):
    """Remove the 'input' key if its value is an empty dictionary."""
    if isinstance(data, dict):
        # Remove 'input' key if its value is {}
        return {k: remove_empty_input(v) for k, v in data.items() if not (k == "input" and v == {})}
    elif isinstance(data, list):
        # Apply the function recursively for each item in the list
        return [remove_empty_input(item) for item in data]
    else:
        # Return the data as is if it's neither a dict nor a list
        return data

In [12]:
def final_json_files(input_folder):
    """Process all JSON files in the given folder to remove 'input': {}."""
    for filename in os.listdir(input_folder):
        if filename.endswith(".json"):
            file_path = os.path.join(input_folder, filename)
            
            # Read the JSON data from the file
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # Remove 'input': {} from the JSON data
            cleaned_data = remove_empty_input(data)
            
            # Save the cleaned data back to the file
            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(cleaned_data, f, ensure_ascii=False, indent=4)

In [13]:
input_folder = "json_files"  # Replace with your folder path
final_json_files(input_folder)

In [14]:
input_folder = "similar_profiles"  # Replace with your folder path
final_json_files(input_folder)